In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StringType
from pyspark.sql.functions import pandas_udf
import pandas as pd
import numpy as np
import pickle

In [2]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [3]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://dlcdn.apache.org/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz
    !tar xf spark-3.5.5-bin-hadoop3.tgz
    !mv spark-3.5.5-bin-hadoop3 spark
    !pip install -q findspark
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark"

In [4]:
import findspark
findspark.init()
spark_url = 'local'
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark')\
        .getOrCreate()

---

In [ ]:
# Loading File from Google Drive
!gdown 'https://drive.google.com/uc?id=1PE-fNRsLA9qqEHmaiof0xV-Uz_5ZfCuO' --output df_with_cluster.csv

In [17]:
# -------------- TABLE 1: Showing Example Comment ---------------------

# 1. Read the CSV
df_with_clustering = spark.read.csv('df_with_cluster.csv', header=True, inferSchema=True)

# 2. Select and rename
df = df_with_clustering.select(
    col('cluster_id').alias('cluster'),
    col('comment')
)

# 3. Drop noise row
df = df.filter(~col('cluster').startswith('noise'))

# 4. Save the DataFrame as a single CSV file
df.coalesce(1).write.csv('example_comment', header=True, mode='overwrite')

---

In [ ]:
!gdown 'https://drive.google.com/uc?id=12RR873XXJrFnJiKJWkqYNnzAfUB-sDyL' --output bright.csv
!gdown 'https://drive.google.com/uc?id=1oAjRZe9Mq80AyLi5G2HSwsXxSKY8mZ2C' --output piti.csv

In [ ]:
# -------------- TABLE 2: Map and Graph visualization ---------------------

# 1. piti: 'cluster_id', 'num_times', 'status'
# 2. bright: 'cluster_id', 'cluster_desc', 'lat', 'long', 'organization'
# 3. keen: zone scraping to get 'zone'

# 1. Load the pre-trained KNN model from Google Drive
!gdown --fuzzy "https://drive.google.com/file/d/1w21nVLe-xYaQ3ZiP5nOY_f9R_t9dMbII/view?usp=drive_link"

with open("knn.pkl", "rb") as f:
    knn = pickle.load(f)

# 2. Read the CSV
piti_df = spark.read.csv('piti.csv', header=True, inferSchema=True)
bright_df = spark.read.csv('bright.csv', header=True, inferSchema=True)
bright_df = bright_df.withColumnRenamed("lon", "long")
bright_df = bright_df.filter(~col('cluster_id').startswith('noise'))

# 3. Merge the tables on 'cluster_id'
merged_df = piti_df.join(bright_df, on='cluster_id', how='inner')

# 4. Define the pandas_udf function to apply the KNN model and predict the zone
@pandas_udf(StringType())
def predict_zone_pandas_udf(lat: pd.Series, lon: pd.Series) -> pd.Series:
    # Assuming knn is already loaded with pickle
    coords = np.column_stack((lat.values, lon.values))  # Combine lat and lon into a 2D array
    return pd.Series(knn.predict(coords))

# 5. Apply the pandas_udf function to predict the zone based on latitude and longitude
merged_df = merged_df.withColumn('zone', predict_zone_pandas_udf(col('lat'), col('long')))

# 6. Save the DataFrame as a single CSV file
merged_df.coalesce(1).write.csv('cluster_data', header=True, mode='overwrite')